### step1 获取月时间和日期（202304）
### step2 获取文件中含有当月日期的excel
### step3 读取该文件到pandas

In [29]:
import pandas as pd
import os
import datetime
import re


In [30]:
current_date = datetime.datetime.now().strftime('%Y%m')#获取当前年月格式2023xx
# monthfirstday = datetime.datetime.now().strftime('%Y%m%d')
# monthfirstday
today = datetime.datetime.today()
# monthfirstday = datetime.datetime(today.year, today.month, 1, 0, 0, 0).strftime('%Y%m%d %H:%M:%S')
# monthfirstday
# alist = [monthfirstday]
# alist

In [31]:
path1 = r"C:\Users\hank\PythonCode\Monthlybill"
files = os.listdir(path1)


In [32]:
pattern = 'CHW RTU_Monthly Report_'+current_date
pattern

'CHW RTU_Monthly Report_202304'

In [33]:
for f in files:
    if pattern in f :
        path = f
        print(path)
# 模糊查找，文件夹下含有'CHW RTU_Monthly Report_'+current_date的文件

CHW RTU_Monthly Report_20230401000853822.xlsx


In [34]:
filepath = path1+"\\"+path
path0 = r"C:\Users\hank\PythonCode\Monthlybill\CHW RTU_Monthly Report_20230401000853822.xlsx"
print(path0)
print(filepath)


C:\Users\hank\PythonCode\Monthlybill\CHW RTU_Monthly Report_20230401000853822.xlsx
C:\Users\hank\PythonCode\Monthlybill\CHW RTU_Monthly Report_20230401000853822.xlsx


In [35]:
df = pd.read_excel(filepath,
                   sheet_name='Report',
                   header=1,
                   parse_dates=['datetime']
                   )
df.head()
# x = [10,104]
# df.drop(df.columns[x],axis=1,inplace=True)
# df

,datetime,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU,D1_FT_1110,...,OF_L_LD2,OF_L_LD2_Counter,SA_H_LD1,SA_H_LD1_Counter,SA_H_LD2,SA_H_LD2_Counter,SA_L_LD1,SA_L_LD1_Counter,SA_L_LD2,SA_L_LD2_Counter
0,2023-02-28 23:00:00,kWH,kWH,kWH,kWH,kWH,kWH,kWH,kWH,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-01 00:00:00,55496884,46414976,22884310,6300717,10043060,2421921,5881492,1188026,79.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-03-01 01:00:00,55496936,46415020,22884464,6300745,10043060,2421921,5881492,1188026,79.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-03-01 02:00:00,55496972,46415032,22884654,6300766,10043097,2421930,5881492,1188026,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-03-01 03:00:00,55496996,46415032,22884826,6300785,10043144,2421933,5881492,1188026,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df.drop(df.columns[9:105],axis=1,inplace=True)#删除后面的列
df.head()

,datetime,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU
0,2023-02-28 23:00:00,kWH,kWH,kWH,kWH,kWH,kWH,kWH,kWH
1,2023-03-01 00:00:00,55496884,46414976,22884310,6300717,10043060,2421921,5881492,1188026
2,2023-03-01 01:00:00,55496936,46415020,22884464,6300745,10043060,2421921,5881492,1188026
3,2023-03-01 02:00:00,55496972,46415032,22884654,6300766,10043097,2421930,5881492,1188026
4,2023-03-01 03:00:00,55496996,46415032,22884826,6300785,10043144,2421933,5881492,1188026


In [37]:
# df.rename(columns={'2023-04-01 00:00:00':'time&date'},inplace=True)
# df.columns = list(["date&time",    'D1_QQ_1150_BTU',    'D2_QQ_1150_BTU',
#           'D3_QQ_1150_BTU',    'D4_QQ_1150_BTU',    'D5_QQ_1150_BTU',
#           'D6_QQ_1150_BTU',    'D7_QQ_1150_BTU',    'D8_QQ_1150_BTU'])#重命名列
df.drop(df.index[0],inplace=True)#删除第一行
df.head()

,date&time,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU
1,2023-03-01 00:00:00,55496884,46414976,22884310,6300717,10043060,2421921,5881492,1188026
2,2023-03-01 01:00:00,55496936,46415020,22884464,6300745,10043060,2421921,5881492,1188026
3,2023-03-01 02:00:00,55496972,46415032,22884654,6300766,10043097,2421930,5881492,1188026
4,2023-03-01 03:00:00,55496996,46415032,22884826,6300785,10043144,2421933,5881492,1188026
5,2023-03-01 04:00:00,55497024,46415032,22885050,6300807,10043150,2421933,5881492,1188026


In [38]:
# df.groupby(pd.Grouper(key='date&time', axis=0, 
#                       freq='D')).max()


1     2023-03-01 00:00:00
2     2023-03-01 01:00:00
3     2023-03-01 02:00:00
4     2023-03-01 03:00:00
5     2023-03-01 04:00:00
              ...        
741   2023-03-31 20:00:00
742   2023-03-31 21:00:00
743   2023-03-31 22:00:00
744   2023-03-31 23:00:00
745   2023-04-01 00:00:00
Name: date&time, Length: 745, dtype: datetime64[ns]